### Llibreries

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import pandas as pd
import joblib

from joblib import dump
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

### Funcions

Inicialitzem i guardem les funcions per fer el face mesh. El 'index_lips' són tots els punts que defineixen els llavis.

In [2]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
index_lips = [61, 76, 62, 78, 
              185, 184, 183, 191, 95, 96, 77, 146, 
              40, 74, 42, 80, 88, 89, 90, 91, 
              39, 73, 41, 81, 178, 179, 180, 181, 
              37, 72, 38, 82, 87, 86, 85, 84,
              0, 11, 12, 13, 14, 15, 16, 17,
              267, 302, 268, 312, 317, 316, 315, 314, 
              269, 303, 271, 311, 402, 403, 404, 405, 
              270, 304, 272, 310, 318, 319, 320, 321, 
              409, 408, 407, 415, 324, 325, 307, 375, 
              308, 292, 306, 291]

## Vowel Recognition solament amb 'face_mesh'

### Creació del dataset

In [3]:
carpeta_principal = 'data/img/'
vocales_info = []
img_lip_info = []
vocales_info_b = []
img_lip_info_b = []

for entrada in os.listdir(carpeta_principal):
    ruta_absoluta = os.path.join(carpeta_principal, entrada)
    # Verifica si la entrada es una carpeta
    if os.path.isdir(ruta_absoluta):
        # Ruta de la carpeta que contiene las imágenes
        carpeta = 'data/img/' + entrada

    # Itera sobre todos los archivos en la carpeta
    for archivo in os.listdir(carpeta):
        # Verifica si el archivo es una imagen (puedes ajustar esta condición según el tipo de imágenes que tengas)
        if archivo.endswith('.jpg') or archivo.endswith('.png') or archivo.endswith('.jpeg'):
            ruta_imagen = os.path.join(carpeta, archivo)
        
            with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5) as face_mesh:
                lip_info = []
                lip_info_b = []
                image = cv2.imread(ruta_imagen)
                height, width, _ = image.shape
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = face_mesh.process(image_rgb)

                if results.multi_face_landmarks is not None:
                    for face_landmarks in results.multi_face_landmarks:
                        for index in index_lips:
                            #lip_info.append(face_landmarks.landmark[index])
                            lip_info.append([face_landmarks.landmark[index].x, face_landmarks.landmark[index].y, face_landmarks.landmark[index].z])
                            lip_info_b.append([face_landmarks.landmark[index].x, face_landmarks.landmark[index].y])

                pos_x = []
                pos_y = []
                pos_z = []
                for i in lip_info:
                    pos_x.append(i[0])
                    pos_y.append(i[1])
                    pos_z.append(i[2])

                aux_x = np.mean(pos_x)
                aux_y = np.mean(pos_y)
                aux_z = np.mean(pos_z)
                lip_info.append([aux_x, aux_y, aux_z])

                pos_x = []
                pos_y = []
                for i in lip_info_b:
                    pos_x.append(i[0])
                    pos_y.append(i[1])

                aux_x = np.mean(pos_x)
                aux_y = np.mean(pos_y)
                lip_info_b.append([aux_x, aux_y])
        
        img_lip_info.append(lip_info)
        img_lip_info_b.append(lip_info_b)
    vocales_info.append(img_lip_info)
    vocales_info_b.append(img_lip_info_b)
    img_lip_info = []
    img_lip_info_b = []

In [4]:
len_file = len(vocales_info[0])
len_file

251

In [5]:
len_vi = len(vocales_info)
len_vi

20

In [6]:
for i in vocales_info:
    print(len(i))

251
137
132
241
218
137
144
239
268
137
152
236
318
137
143
227
311
137
157
253


In [7]:
min = 1000
for i in vocales_info:
    if (len(i) < min) and (len(i) > 0):
        min = len(i)
        
for i in range(len(vocales_info)):
    vocales_info[i] = vocales_info[i][:min]
    vocales_info_b[i] = vocales_info_b[i][:min]
    
for i in vocales_info:
    print(len(i))

132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132


In [8]:
min

132

In [9]:
vocales_info_np = np.array(vocales_info)
vocales_info_np_b = np.array(vocales_info_b)

In [10]:
coordenada_central = 80
vocales_info_2 = []
img_lip_info_2 = []
for i in range(len(vocales_info_np)):
    for j in range(len(vocales_info_np[i])):
        lip_info_2 = []
        for k in range(len(vocales_info_np[i][j]) - 1):
            distancia = np.linalg.norm(vocales_info_np[i][j][k] - vocales_info_np[i][j][coordenada_central])
            lip_info_2.append(distancia)
        img_lip_info_2.append(lip_info_2)
    vocales_info_2.append(img_lip_info_2)
    img_lip_info_2 = []

In [11]:
vocales_info_2[0][0][0]

0.03857905151367015

In [12]:
coordenada_central = 80
vocales_info_b_2 = []
img_lip_info_b_2 = []
for i in range(len(vocales_info_np_b)):
    for j in range(len(vocales_info_np_b[i])):
        lip_info_b_2 = []
        for k in range(len(vocales_info_np_b[i][j]) - 1):
            distancia = np.linalg.norm(vocales_info_np_b[i][j][k] - vocales_info_np_b[i][j][coordenada_central])
            lip_info_b_2.append(distancia)
        img_lip_info_b_2.append(lip_info_b_2)
    vocales_info_b_2.append(img_lip_info_b_2)
    img_lip_info_b_2 = []

In [13]:
vocales_info_b_2[0][0][0]

0.03433913851860893

In [14]:
# Crear un DataFrame de Pandas a partir del arreglo
#arreglo_aplanado = [valor for sublist in vocales_info for valor in sublist]
#df = pd.DataFrame(arreglo_aplanado)

#df

In [15]:
longitud_array_vocals = min*len_vi
aux_a_v = 0

array_vocals = []
for i in range(longitud_array_vocals):
    if i % (longitud_array_vocals/5) == 0:
        aux_a_v += 1  
    array_vocals.append(aux_a_v)

df_array_vocals = pd.DataFrame(array_vocals, columns=['Y'])
df_array_vocals

,Y
0,1
1,1
2,1
3,1
4,1
...,...
2635,5
2636,5
2637,5
2638,5


In [16]:
df_1 = pd.DataFrame()
for i in range(len(vocales_info_2)):
    df_aux = pd.DataFrame(vocales_info_2[i])
    df_1 = pd.concat([df_1, df_aux])

In [17]:
df_1

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
0,0.038579,0.035747,0.033045,0.031771,0.038828,0.035187,0.031465,0.030538,0.027221,0.029642,...,0.029540,0.028635,0.028820,0.031299,0.033967,0.036688,0.031642,0.032878,0.035329,0.037753
1,0.039934,0.037168,0.034478,0.033183,0.040386,0.036717,0.032977,0.032056,0.027885,0.030229,...,0.030614,0.029639,0.030238,0.032706,0.035389,0.038118,0.033223,0.034519,0.036997,0.039378
2,0.038330,0.035573,0.032937,0.031740,0.038501,0.034874,0.031193,0.030328,0.026804,0.029079,...,0.029150,0.028207,0.028834,0.031282,0.033986,0.036794,0.031832,0.033066,0.035499,0.037876
3,0.037575,0.034756,0.032081,0.030880,0.037512,0.033834,0.030129,0.029221,0.026354,0.028668,...,0.028453,0.027502,0.027980,0.030425,0.033183,0.036069,0.030960,0.032144,0.034594,0.037029
4,0.038761,0.035986,0.033310,0.032068,0.039005,0.035338,0.031636,0.030752,0.027242,0.029589,...,0.029592,0.028647,0.029208,0.031678,0.034404,0.037196,0.032093,0.033374,0.035865,0.038271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,0.089939,0.082300,0.075548,0.073151,0.080977,0.072858,0.064383,0.059482,0.063001,0.067531,...,0.066538,0.061592,0.064225,0.068740,0.075332,0.083302,0.075106,0.078096,0.085038,0.093249
128,0.088840,0.081073,0.074203,0.071793,0.080132,0.071819,0.063144,0.058173,0.061844,0.066462,...,0.064771,0.059819,0.062679,0.067241,0.073860,0.081867,0.073037,0.075942,0.082949,0.091241
129,0.088729,0.080952,0.074026,0.071618,0.079899,0.071635,0.062965,0.057952,0.061730,0.066427,...,0.064676,0.059707,0.062702,0.067263,0.073953,0.082025,0.073100,0.076025,0.083126,0.091541
130,0.090670,0.083091,0.076415,0.074092,0.081644,0.073588,0.065207,0.060384,0.063564,0.068023,...,0.067061,0.062143,0.064628,0.069036,0.075516,0.083408,0.075646,0.078578,0.085473,0.093679


In [18]:
df_2 = pd.DataFrame()
for i in range(len(vocales_info_b_2)):
    df_aux = pd.DataFrame(vocales_info_b_2[i])
    df_2 = pd.concat([df_2, df_aux])

In [19]:
df_2

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
0,0.034339,0.031953,0.029549,0.028233,0.037570,0.033932,0.030246,0.029138,0.023918,0.026983,...,0.029538,0.028615,0.028383,0.031027,0.033751,0.036425,0.031164,0.032427,0.034779,0.037091
1,0.035356,0.033026,0.030634,0.029283,0.038972,0.035283,0.031583,0.030470,0.024210,0.027224,...,0.030613,0.029623,0.029829,0.032449,0.035181,0.037858,0.032760,0.034078,0.036462,0.038740
2,0.033631,0.031330,0.029006,0.027764,0.037039,0.033384,0.029731,0.028656,0.023113,0.026070,...,0.029150,0.028196,0.028475,0.031068,0.033816,0.036572,0.031423,0.032677,0.035018,0.037296
3,0.033223,0.030854,0.028486,0.027254,0.036206,0.032513,0.028827,0.027709,0.023058,0.026018,...,0.028451,0.027477,0.027569,0.030177,0.032987,0.035822,0.030488,0.031693,0.034043,0.036361
4,0.034355,0.032009,0.029635,0.028342,0.037664,0.033984,0.030308,0.029228,0.023798,0.026798,...,0.029589,0.028620,0.028775,0.031411,0.034191,0.036929,0.031604,0.032905,0.035296,0.037594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,0.084777,0.078006,0.072055,0.069642,0.080109,0.071947,0.063251,0.057718,0.060900,0.066245,...,0.064733,0.059071,0.061061,0.066700,0.073623,0.080977,0.070372,0.073349,0.079403,0.086517
128,0.083994,0.077068,0.070986,0.068541,0.079424,0.071071,0.062185,0.056614,0.059883,0.065267,...,0.063014,0.057325,0.059428,0.065125,0.072098,0.079478,0.068274,0.071181,0.077272,0.084436
129,0.083844,0.076903,0.070786,0.068361,0.079199,0.070902,0.062024,0.056406,0.059784,0.065243,...,0.062866,0.057150,0.059403,0.065102,0.072130,0.079553,0.068246,0.071179,0.077359,0.084637
130,0.085572,0.078848,0.072994,0.070671,0.080803,0.072709,0.064113,0.058665,0.061578,0.066813,...,0.065169,0.059513,0.061444,0.066961,0.073758,0.081003,0.070801,0.073722,0.079701,0.086784


### Entrenament del model Support Vector Machine

In [20]:
# Definir los parámetros a ajustar
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1], 'kernel': ['linear', 'rbf', 'poly']}

# Realizar la búsqueda en cuadrícula para encontrar la mejor combinación de hiperparámetros
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(df_1, df_array_vocals.values.ravel())

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)

Mejores hiperparámetros: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}


In [21]:
# Definir los parámetros a ajustar
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1], 'kernel': ['linear', 'rbf', 'poly']}

# Realizar la búsqueda en cuadrícula para encontrar la mejor combinación de hiperparámetros
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(df_2, df_array_vocals.values.ravel())

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)

Mejores hiperparámetros: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}


In [22]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df_1, df_array_vocals.values.ravel(), test_size=0.2, random_state=42)

# Crear un clasificador SVM
svm_classifier_1 = SVC(kernel='rbf', C=100, gamma=1)

# Entrenar el clasificador
svm_classifier_1.fit(X_train, y_train)

# Predecir las etiquetas de clase para el conjunto de prueba
y_pred = svm_classifier_1.predict(X_test)

# Calcular la precisión del clasificador
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9223484848484849


In [23]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df_2, df_array_vocals.values.ravel(), test_size=0.2, random_state=42)

# Crear un clasificador SVM
svm_classifier_2 = SVC(kernel='rbf', C=100, gamma=1)

# Entrenar el clasificador
svm_classifier_2.fit(X_train, y_train)

# Predecir las etiquetas de clase para el conjunto de prueba
y_pred = svm_classifier_2.predict(X_test)

# Calcular la precisión del clasificador
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9261363636363636


In [26]:
# Guardar el modelo
dump(svm_classifier_1, 'models/svm/svm_classifier_a.joblib')
dump(svm_classifier_2, 'models/svm/svm_classifier_b.joblib')

['models/svm/svm_classifier_b.joblib']

In [27]:
# Realizar validación cruzada para evaluar el rendimiento del modelo SVM
scores = cross_val_score(SVC(), df_1, df_array_vocals.values.ravel(), cv=5)
print("Precisión de validación cruzada:", scores.mean())

Precisión de validación cruzada: 0.6886363636363637


In [28]:
# Realizar validación cruzada para evaluar el rendimiento del modelo SVM
scores = cross_val_score(SVC(), df_2, df_array_vocals.values.ravel(), cv=5)
print("Precisión de validación cruzada:", scores.mean())

Precisión de validación cruzada: 0.5973484848484849


### Entrenament del model Support Vector Machine

In [24]:
X_train, X_test, y_train, y_test = train_test_split(df_1, df_array_vocals.values.ravel(), test_size=0.2, random_state=42)

rf_classifier_1 = RandomForestClassifier(n_estimators=1000, random_state=42)

rf_classifier_1.fit(X_train, y_train)

y_pred = rf_classifier_1.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9943181818181818


In [25]:
X_train, X_test, y_train, y_test = train_test_split(df_2, df_array_vocals.values.ravel(), test_size=0.2, random_state=42)

rf_classifier_2 = RandomForestClassifier(n_estimators=1000, random_state=42)

rf_classifier_2.fit(X_train, y_train)

y_pred = rf_classifier_2.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9943181818181818


In [29]:
# Guardar el modelo
dump(rf_classifier_1, 'models/rf/rf_classifier_a.joblib')
dump(rf_classifier_2, 'models/rf/rf_classifier_b.joblib')

['models/rf/rf_classifier_b.joblib']